In [ ]:
#https://levelup.gitconnected.com/deploy-simple-and-instant-online-quizzes-with-jupyter-notebook-tools-5e10f37da531
#Run this via "voila" in the browser

In [1]:
#%%bash
#conda install pandas

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py38h578d9bd_1         143 KB  conda-forge
    libblas-3.9.0              |       7_openblas          11 KB  conda-forge
    libcblas-3.9.0             |       7_openblas          11 KB  conda-forge
    libgfortran-ng-9.3.0       |      he4bcb1c_17          22 KB  conda-forge
    libgfortran5-9.3.0         |      he4bcb1c_17         2.0 MB  conda-forge
    liblapack-3.9.0            |       7_openblas          11 KB  conda-forge
    libopenblas-0.3.12         |pthreads_h4812303_1         8.9 MB  conda-forge
    numpy-1.19.5               |   py38h18fd61f_1         5.4 MB  conda-forge
    pandas-1.2.0               |   py38h51da96c_1        12.0 MB  conda-forge
    pytz-2020.5

In [2]:
import ipywidgets as widgets
import sys
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Layout, Box
import ipywidgets
import pandas as pd
from pathlib import Path
from collections import namedtuple
import os
import difflib
import re
import ast

In [3]:
# globally initialize output

output = widgets.Output()

In [4]:
# helper functions

def save_intro_answers(answers):
    file = open('human_eval_agreement.txt',"w")
    file.write('Participant agreed/consented: ')
    file.write(str(answers))
    file.close()
    
def save_further_comments(text):
    file = open('human_eval_agreement.txt',"a")
    file.write('\nFurther comments:\n')
    file.write(text)
    file.write('\nParticipant finished evaluation.')
    file.close

def save_eval_answers(answers_df):
    path = Path('human_eval_answers.csv')
    if os.path.exists(path):
        answers_df.to_csv(path, mode='a', header=False, index=False)
    else:
        answers_df.to_csv(path, header = ['sample_index','query_index','q1', 'q2', 'q3', 'Explanation'],index=False)
        
def clean(text):
    text = re.sub('<([A-Za-z\_]+)((START)|(END))>','',text)
    text = re.sub('<PAR>','',text)
    return text
        
AnswerArray = namedtuple('AnswerArray', ['sample_index','query_index','q1', 'q2', 'q3', 'Explanation'])

In [5]:
# Questionnaire pages (UI)

# FIRST PAGE
def questionnaire_intro(list_of_samples):
    # Intro with description, instructions
    header = widgets.HTML(value = f"<h2>Human Evaluation: Synthetic Text Privacy</h2>")   
    text_intro = '''<i>This research aims to create synthetic text data using a machine learning model \
trained on real patient data. While this synthetic text is meant to share properties with the real\
data to be of use in further research, it should not contain information from the real data that \
could help re-identifying people contained in the real dataset. For example you could ask: If I was \
a patient mentioned in the real dataset, could one learn something about me by looking at the synthetic \
data?<br> Differently to structured datasets with clearly defined attributes (Name, Date, Diagnosis...), \
free text data is more complicated and harder to evaluate, as privacy sensitive information can be disclosed \
via context or different phrasing. As machine-calculated similarity scores are not very indicative of privacy
breaches, it is necessary to have a human evaluate some examples, especially because there is not always a right
or wrong answer.<br><br></i>\

<b>Data:</b> During the evaluation, you will get (1) a synthetic piece of text and (2) a similar text from the real dataset, \
which we present as potential source document for the given synthetic text. There are no true 1:1 matches between \
original and fake texts, so you may get to see the same synthetic text twice, but with different potential source texts.<br><br>\

<b>Questions:</b> You will be asked the same questions for each example. The aim is to better understand whether privacy of \
people in the real dataset is compromised by looking at the synthetic data. Note that we do NOT care about how \
realistic/grammatical the synthetic texts are. Please read each text carefully. It is up to you to decide whether \
you consider certain information as privacy sensitive, as there is no right or wrong answer.<br><br>'''
    intro = widgets.HTML(value = f"<p><font size=3>{text_intro}</p>")
    contactinfo = widgets.HTML(value = f"<p><font size=3><font color='blue'>For any questions or feedback, please contact me on Slack @claudia.libbi <br></p>")

    # Ethical
    headerethical = widgets.HTML(value = f"<h3>Ethical Approval</h3>")
    text_ethical = '''We did a DPIA (Data Protection Impact Assessment) with the Privacy Officer at Nedap. <br>\
The data that will be shown to you is privacy sensitive and may be used within this research project and can \
not be shared with any third person.<br>'''
    ethical = widgets.HTML(value = f"<p><font size=3>{text_ethical}</p>")
    checkbox1 = widgets.Checkbox(
        value = False,
        description = 'I understand that I may not share this data with anyone else.',
        disabled = False,
        indent = False,
        layout = Layout(width='100%')
    )
        
    # Confidentiality
    headerconf = widgets.HTML(value = f"<h3>Confidentiality</h3>")
    text_conf = '''Your answers will be treated confidentially and stored anonymously for the duration of \
this study, as we do not need to re-identify you as evaluator after data collection.<br>\
Your name will not be mentioned in any publications resulting from this research unless you explicitly \
consent to this. <br>'''
    conf = widgets.HTML(value = f"<p><font size=3>{text_conf}</p>")
    checkbox2 = widgets.Checkbox(
        value = False,
        description = 'I understand that my answers will be treated confidentially and will be stored anonymously for the duration of this research.',
        disabled = False,
        indent = False,
        layout=Layout(width='100%')
    )
    
    # Next button
    def save_selection(b):
        answer1 = checkbox1.value
        answer2 = checkbox2.value
        if answer1 == False or answer2 == False:
            warning = widgets.HTML(value = f"<p><font size=3><font color='red'>You can only continue if you tick both boxes.</p>")
            with output:
                display(warning)
        else:
            save_intro_answers([answer1, answer2])
            with output:
                output.clear_output(wait=True)
                sample_and_display_example(list_of_samples) 
    save = widgets.Button(description="Next")
    save.on_click(save_selection)
    
    return widgets.VBox([header,intro,contactinfo,headerethical,ethical,checkbox1, headerconf, conf, checkbox2,save])


#CENTRAL: next example or finish ====================================================================================
def sample_and_display_example(list_of_samples):    
    if list_of_samples:
        next_sample = list_of_samples[0]
        list_of_samples = list_of_samples[1:]
        with output:
            display(questionnaire_page(next_sample[0],next_sample[1],next_sample[2],next_sample[3],next_sample[4],list_of_samples))
    else:
        with output:
            display(last_page())


#FINISH: extra comments ====================================================================================
def last_page():
    instructions = widgets.HTML(value = f"<p><font size=3>You have completed this evaluation, thank you for your help!</p>")
    prompt_text = 'If you have feedback or comments for this study, please indicate it below.'
    prompt = widgets.HTML(value = f"<p><font size=3>{prompt_text}</p>")
    text_out = widgets.Textarea(
        value='',
        placeholder='',
        disabled=False
    )
    def save_selection(b):
        save_further_comments(text_out.value)
        with output:
            clear_output(wait=True)
            display(exit())
        return
    
    save = widgets.Button(description="Finish")
    save.on_click(save_selection)
    
    return widgets.VBox([instructions,prompt,text_out,save])


#FINISH: close tab screen ==================================================================================
def exit():
    finaltext = widgets.HTML(value = f"<p><font size=3>'You can now close the tab :)'</p>")
    
    return widgets.VBox([finaltext])


#SHOW EXAMPLE WITH QUESTION ====================================================================================
def questionnaire_page(sample_index,query_index,S,RC,difflist,list_of_samples):
    # highlight differences
    
    #difflist = ast.literal_eval(difflist)
    S = clean(S)
    RC = clean(RC)
    #for chunk in difflist:
        #S = S.replace(chunk,'<b><font color=blue>{}</font color=blue></b>'.format(chunk))
        #RC = RC.replace(chunk,'<b><font color=blue>{}</font color=blue></b>'.format(chunk))
    
    # display real candidate1
    rc = widgets.HTML(value = f"<b><font size=3>Real doc:</b>")
    rc_text = widgets.HTML(value = f"<p>{RC}</p>")
    Q1 = widgets.HTML(value = f"<b><font size=3>Do you think the real doc provides enough information to identify a person?</b>")
    Q1_options = widgets.ToggleButtons(
        options = ['(Select)','Yes','Probably','Not sure','Probably not','No'],
        description = '',
        disabled = False,
    )
    
    # display synthetic
    s = widgets.HTML(value = f"<b><font size=3><br>Synthetic doc:</b>")
    s_text = widgets.HTML(value = f"<p>{S}</p>")
    
    Q2 = widgets.HTML(value = f"<b><font size=3>Do you think the synthetic doc contains person identifying information?</b>")
    Q2_options = widgets.ToggleButtons(
        options = ['(Select)','Yes','Probably','Not sure','Probably not','No'],
        description = '',
        disabled = False,
    )
    
    #Question
    Q3 = widgets.HTML(value = f"<b><font size=3><br>Do you think that there is a link between the synthetic and real doc in the sense that it may identify someone in the real doc?</b>")
    Q3_options = widgets.ToggleButtons(
        options = ['(Select)','Yes','Probably','Not sure','Probably not','No'],
        description = '',
        disabled = False,
    )
    Q3prompt = widgets.HTML(value = f"<p><font size=3>Please explain your choice:</p>")
    Q3_textopt = widgets.Textarea(
        value='',
        placeholder='',
        disabled=False
    )
    
    def save_selection(b):
        q1 = Q1_options.value
        q2 = Q2_options.value
        q3 = Q3_options.value
        q3t = Q3_textopt.value
        # check if all obligatory fields are answered, save and move on.
        if (q1!='(Select)' and q2!='(Select)' and q3!='(Select)' and q3t):
            # Save answers to outfilepath with sampleindex
            answers_df = pd.DataFrame(AnswerArray(sample_index,query_index, q1, q2, q3, q3t)).T
            save_eval_answers(answers_df)
            #clear display and move on
            with output:
                clear_output(wait=True)
                sample_and_display_example(list_of_samples)
        else:
            warning = widgets.HTML(value = f"<p><font size=3><font color='red'>Please answer all questions to continue.</p>")
            with output:
                display(warning)
        return
    
    save = widgets.Button(description="Next")
    save.on_click(save_selection)
    
    return widgets.VBox([rc, rc_text, Q1,Q1_options,s,s_text,Q2,Q2_options,Q3,Q3_options,Q3prompt,Q3_textopt,save])

In [7]:
def run():
    
    # get input file as df ==========================================================
    
    df = pd.read_csv(Path('examples4participantX.csv'),index_col=0)[['query_index','query','result','overlap']]
    
    # create list of samples to display =============================================
    # check if started...
    outfilepath = Path('human_eval_answers.csv')
    if os.path.exists(outfilepath):
        start_idx = len(pd.read_csv(outfilepath))
    else:
        start_idx = 0
        
    list_of_samples = list(df.itertuples(index=True, name=None))[start_idx:]
    
    with output:
        display(questionnaire_intro(list_of_samples))
    display(output)

In [58]:
run()

Output()